In [ ]:
import boto3
import json

In [ ]:
codepipeline = boto3.client('codepipeline')
sm = boto3.client('sagemaker-runtime')

pipeline_name = 'mlops-iris-model'
endpoint_name_mask='mlops-iris-model-%s-%s'

In [ ]:
def get_actions():
    actions = []
    executionId = None
    resp = codepipeline.get_pipeline_state( name=pipeline_name )
    for stage in resp['stageStates']:
        stageName = stage['stageName']
        stageStatus = None
        if stage.get('latestExecution') is not None:
            stageStatus = stage['latestExecution']['status']
            if executionId is None:
                executionId = stage['latestExecution']['pipelineExecutionId']
            elif stage['latestExecution']['pipelineExecutionId'] != executionId:
                stageStatus = 'Old'
        for action in stage['actionStates']:
            actionName = action['actionName']
            actionStatus = 'Old'
            if action.get('latestExecution') is not None and stageStatus != 'Old':
                actionStatus = action['latestExecution']['status']
            actions.append( {'stageName': stageName, 
                             'stageStatus': stageStatus, 
                             'actionName': actionName, 
                             'actionStatus': actionStatus})
    return actions

In [ ]:
def get_approval_token():
    resp = codepipeline.get_pipeline_state( name=pipeline_name )
    token = None
    # Get the approve train status token
    for stageState in resp['stageStates']:
      if stageState['stageName'] == 'TrainApproval':
        for actionState in stageState['actionStates']:
          if actionState['actionName'] == 'ApproveTrain':
            latestExecution = actionState['latestExecution']
            if latestExecution['status'] == 'InProgress':
                token = latestExecution['token']
    return token

In [ ]:
def monitor_job():
    print('TODO')

In [ ]:
def test_endpoint(endpoint_name, payload):
    resp = sm.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=payload
    )
    return resp['Body'].read()

In [ ]:
response = codepipeline.get_pipeline_state( name=pipeline_name )
executionId = response['stageStates'][0]['latestExecution']['pipelineExecutionId']

In [ ]:
payload_logistic = json.dumps({"algorithm": "logistic", "payload": [[4.6, 3.1, 1.5, 0.2]] })
payload_random_forest = json.dumps({"algorithm": "random_forest", "payload": [[4.6, 3.1, 1.5, 0.2]] })

In [ ]:
%%time
print( "DSV")
print( "Logistic: %s" % test_endpoint( endpoint_name_mask % ('dev', executionId), payload_logistic ) )
print( "Random forest: %s" % test_endpoint( endpoint_name_mask % ('dev', executionId), payload_random_forest ) )
print( "\nPRD")
print( "Logistic: %s" % test_endpoint( endpoint_name_mask % ('prd', executionId), payload_logistic ) )
print( "Random forest: %s" % test_endpoint( endpoint_name_mask % ('prd', executionId), payload_random_forest ) )